In [91]:
# use OOT data to estimate bkg in signal region

import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2


# donotdelete = []
print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


In [92]:
def keyToyear(key):
    for year in years:
        if year in key:
            return year

# Load ntuples

In [107]:
fpath =OrderedDict()
tree = OrderedDict()
mass = [15, 40, 55]
mass = [15]
# ctau = [1, 10, 100, 1000, 10000, 100000]
# ctau = [10, 100, 1000, 10000, 100000]
OLD_CTAU = np.array([100, 1000, 10000, 100000])#in mm


years = ['Summer16', 'Fall17', 'Fall18']
lumi = {
    'Summer16': 35.92 * 1000,
    'Fall17': 41.53 * 1000,
    'Fall18': 59.74 * 1000,
}
category = 0
ntupler_version = 'V1p15/'
analyzer_version = 'v3/v4/'

data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p15/Data2018/v1/v1/normalized/'

fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p15_Data2016_Data2017_Data2018-HighMET_goodLumi.root'
for y in years:
        mc_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_'+y+'/'+analyzer_version+'/normalized/'
        tune = 'TuneCP5'
        if y == 'Summer16': tune = 'TuneCUETP8M1'
        for m in mass:
                for ct in OLD_CTAU:
                        key = 'MC_'+y+'_ggH_'+str(m)+'_'+str(ct)                       
                        fpath[key] = mc_path+'ggH_HToSSTobbbb_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_' + tune + '_13TeV-powheg-pythia8_1pb_weighted.root'
                        key = 'MC_'+y+'_VBFH_'+str(m)+'_'+str(ct)                       
                        fpath[key] = mc_path+'VBFH_HToSSTobbbb_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_' + tune + '_13TeV-powheg-pythia8_1pb_weighted.root'
#                         key = 'MC_'+y+'_VBFH_'+str(m)+'_'+str(ct)                       
#                         fpath[key] = mc_path+'VBFH_HToSSTobbbb_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_'+tune+'_13TeV-powheg-pythia8_1pb_weighted.root'

NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    print (k, v)
    root_dir = uproot.open(v) 
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    a = tree[k]["weight"].array()



data /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p15/Data2018/v1/v1/normalized/Run2_displacedJetMuonNtupler_V1p15_Data2016_Data2017_Data2018-HighMET_goodLumi.root
MC_Summer16_ggH_15_100 /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p15//MC_Summer16/v3/v4//normalized/ggH_HToSSTobbbb_MH-125_MS-15_ctau-100_TuneCUETP8M1_13TeV-powheg-pythia8_1pb_weighted.root
MC_Summer16_VBFH_15_100 /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p15//MC_Summer16/v3/v4//normalized/VBFH_HToSSTobbbb_MH-125_MS-15_ctau-100_TuneCUETP8M1_13TeV-powheg-pythia8_1pb_weighted.root
MC_Summer16_ggH_15_1000 /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p15//MC_Summer16/v3/v4//normalized/ggH_HToSSTobbbb_MH-125_MS-15_ctau-1000_TuneCUETP8M1_13TeV-powheg-pythia8_1pb_weighted.root
MC_Summer16_VBFH_15_1000 /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p15//MC_

# Load trigger

In [108]:
trigger_names_file = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/trigger_names_llp_v1.dat'
trigger_names = []
with open(trigger_names_file) as f:
    reader = csv.reader(f, delimiter=" ")
    for line in reader:
        trigger_names.append(line[2])

trigger_paths = [177,362,87,135,136] #PFMET120
trigger_paths += [84,91]
if category == 0:
    trigger_paths = [310]

# load bdt

In [109]:
# make predictions for test data

import pickle
# model = pickle.load(open( 'bdt_flatten_noEvtSelection.pickle.dat', "rb" ))
bdt_name = 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi_v2'
model = pickle.load(open( bdt_name+'.pickle', "rb" ))
y_pred = model.predict_proba([1,2,3,4,4,5,6])[:, 1]
print (y_pred)


[0.24800783]


In [110]:
import math
def deltaPhi( phi1,  phi2):
    dphi = phi1-phi2
    while (dphi > math.pi):
        dphi -= 2*math.pi
    while (dphi <= -math.pi):
        dphi += 2*math.pi
    return dphi
def deltaR(eta1, phi1, eta2, phi2):
    dphi = deltaPhi(phi1,phi2)
    deta = eta1 - eta2
    return (dphi*dphi + deta*deta)**0.5

# nCsc with different hit vetoing

In [111]:
%%time

start_t = time.time()
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False


# BDT_CUT = 0.467 #90% wp vBDT/v1, bdt_flatten_noEvtSelection
# bdtBkgEff =  0.0661076084960519

# BDT_CUT = 0.9227871 #90% wp vBDT/v2, bdt_flatten_metfilter_me1112nohits_eta2p1
# bdtBkgEff =  0.10278372591006424
if bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs':
    BDT_CUT = 0.92245656 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs
    bdtBkgEff = 0.09957173447537473
elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi':
    BDT_CUT = 0.9608465 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi
    bdtBkgEff = 0.06745182012847965
elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi_v2':
    BDT_CUT = 0.7802247 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi
    bdtBkgEff = 0.061027837259100645
else:
    print('BDT NAME NOT FOUND')

intime = True
DPHI_CUT = 1
weight = {}
weight_event = {}
weight_ctau = {}
lumiSec = {}
evtNum = {}
nCsc = {}
npv = {}
npu = {}
runNum = {}
nCsc_JetMuonVetoCluster0p4_Me1112Veto = {}
gLLP_csc = {}
cscClusterSize = {}
cscClusterTime = {}
nCscClusters = {}
selections_cluster = {}
sel_cluster = {}
sel_jetveto = {}
met_trigger = {}
met = {}
gLLP_beta = {}
jetPt = {}
jetPhi = {}
metPhi = {}
angle ={}
nLeptons = {}
genJetPt = {}
genJetPhi = {}
genMet = {}
genMetPhi = {}
pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
cscRechitClusterTimeDiff = {}
cscRechitCluster_match_gLLP = {}

cscRechitClusterXSpread = {}
cscRechitClusterYSpread = {}
cscRechitClusterNStation = {}
cscRechitClusterEtaPhiSpread = {}
cscRechitClusterPhiSpread = {}
cscRechitClusterEtaSpread = {}
cscRechitClusterX = {}
cscRechitClusterY = {}
cscRechitClusterZ = {}
cscRechitClusterPhi = {}
cscClusterJetVetoPt = {}
cscRechitClusterEta = {}

cscRechitClusterMaxStationRatio = {}
cscRechitClusterNStation = {}
cscRechitClusterNChamber = {}
cscRechitClusterMet_dPhi = {}
jetMet_dPhiMin30 = {}
jetMet_dPhiMin = {}
dphiMet_cluster = {}
nRechits_sr = {}
jetMet_dPhiMin30_sr = {}
bdt_score = {}
a = {}
b = {}
c = {}
d = {}
sel_ev = {}
bdt_sel = {}

legend = {}

legend['data_oot'] = 'Data OOT region'
legend['data_intime'] = 'Data in-time region'
legend['mc_signal'] = 'signal MC in time'
legend['mc_intime'] = 'MC in-time background'
legend['mc_oot'] = 'MC OOT background'
legend['mc_bkg'] = 'QCD 50toInf background'

# keys = ['data','mc_intime_bkg','mc_oot_bkg','mc_signal']
keys = ['data_intime','data_oot','mc_signal','mc_bkg']
keys = ['data_intime','mc_signal',]
keys = ['data_intime','mc_signal']
keys = ['data_oot_vr','data_oot_sr', 'data_intime_vr', 'data_intime_sr', 'mc_vr','mc_sr']
keys = ['data_oot_vr','data_oot_sr', 'data_intime_vr', 'data_intime_sr', 'mc100_vr','mc100_sr', 'mc1000_vr', 'mc1000_sr']

# keys = ['mc100_vr','mc100_sr', 'mc1000_vr', 'mc1000_sr']

# for k in keys:
#     print(k)
# #     if k == 'data_intime': continue
#     if k == 'mc_bkg':
#         T = tree_bkg['QCDHT50toInf']
#     elif 'mc1000' in k:
#         T = tree_bkg['mc1000']
#     elif 'mc100' in k:
#         T = tree_bkg['mc100']
#     else:
#         T = tree_bkg['data']
for k, T in tree.items():
#     if k == 'data':continue
########### SELECTION: CLUSTERS ############

    sel_rechitcluster =  np.abs(T.array('cscRechitClusterMaxChamber')) > 12
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('cscRechitClusterEta')) < 2.1)

    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberPlus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberPlus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberMinus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberMinus12') <= me1112_veto)
    if 'oot' in k:
        sel_rechitcluster = np.logical_and(sel_rechitcluster,  T.array('cscRechitClusterTime') < -12.5)
    else:
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitClusterJetVetoPt') < JET_PT_CUT)
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitClusterMuonVetoPt') < MUON_PT_CUT)
        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitClusterTime') < 12.5, T.array('cscRechitClusterTime') > -5.0))
    print(np.count_nonzero(sel_rechitcluster.flatten()))

########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )

########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()
    # select only triggered events
    sel_ev[k] = np.ones(hlt[:,0].shape, dtype=bool)
    sel_ev[k] = np.zeros(hlt[:,0].shape)
    for tr in trigger_paths:            
        sel_ev[k]   = np.logical_or(sel_ev[k] ,hlt[:,tr])
        print("trigger",tr,1.0*np.count_nonzero(hlt[:,tr])/len(hlt[:,tr]))
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('met') > 200)
    sel_ev[k]  = np.logical_and(sel_ev[k], T.array('category') == category)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('nLeptons') == 0)
    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    sel_ev[k]  = np.logical_and(sel_ev[k],sel_rechitcluster.sum() == 1)
    if k[:4] == 'data':
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_HBHENoiseFilter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_HBHEIsoNoiseFilter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_BadPFMuonFilter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_globalSuperTightHalo2016Filter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_EcalDeadCellTriggerPrimitiveFilter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_eeBadScFilter'))
#             sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag_ecalBadCalibFilter'))

        # missing Flag_EcalDeadCellTriggerPrimitiveFilter
#             ecalBadCalibReducedMINIAODFilter
    else:
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_HBHENoiseFilter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_HBHEIsoNoiseFilter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_BadPFMuonFilter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_globalSuperTightHalo2016Filter'))
        sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_EcalDeadCellTriggerPrimitiveFilter'))

#             sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_ecalBadCalibFilter'))
#             sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_eeBadScFilter'))

            

#             sel_ev = np.logical_and(sel_ev, T.array('runNum')<319077)
#             print('selev',np.count_nonzero(sel_ev))
#             sel_ev = np.logical_and(sel_ev, T.array('Flag_ecalBadCalibFilter'))
#             print('selev',np.count_nonzero(T.array('Flag_ecalBadCalibFilter')))
#     gLLP_csc[k] = T.array('gLLP_csc')
#     if k[:2] == 'mc':
#         sel_ev[k] = np.logical_and(sel_ev[k], np.sum(T.array('gLLP_csc'),axis = 1) > 0)
    print(k, len(sel_ev[k]), np.count_nonzero(sel_ev[k]))
########### BRANCHES ############

   ##### event variables ##### 
    
    gLLP_beta[k] = T.array('gLLP_beta')[sel_ev[k]]


    ##### bdt variables ####
    
    cscRechitClusterPhi[k] = T.array('cscRechitClusterPhi')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterXSpread[k] = T.array('cscRechitClusterXSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterYSpread[k] = T.array('cscRechitClusterYSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEtaSpread[k] = T.array('cscRechitClusterEtaSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterPhiSpread[k] = T.array('cscRechitClusterPhiSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterNStation[k] = T.array('cscRechitClusterNStation')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEtaPhiSpread[k] = T.array('cscRechitClusterEtaPhiSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterX[k] = T.array('cscRechitClusterX')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterY[k] = T.array('cscRechitClusterY')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterZ[k] = T.array('cscRechitClusterZ')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEta[k] = T.array('cscRechitClusterEta')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterPhi[k] = T.array('cscRechitClusterPhi')[sel_rechitcluster][sel_ev[k]][:,0].flatten()

    if bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs':
        
        bdt_var = {'cscRechitClusterXSpread': cscRechitClusterXSpread[k],
         'cscRechitClusterYSpread': cscRechitClusterYSpread[k],
         'cscRechitClusterNStation':cscRechitClusterNStation[k],
         'cscRechitClusterEtaPhiSpread':cscRechitClusterEtaPhiSpread[k],
         'cscRechitClusterX':cscRechitClusterX[k],
         'cscRechitClusterY':cscRechitClusterY[k],
         'cscRechitClusterZ':cscRechitClusterZ[k],
        }
    elif 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi' in bdt_name:
        bdt_var = {'cscRechitClusterNStation': cscRechitClusterNStation[k],
         'cscRechitClusterEtaSpread': cscRechitClusterEtaSpread[k],
         'cscRechitClusterPhiSpread':cscRechitClusterPhiSpread[k],
         'cscRechitClusterEtaPhiSpread':cscRechitClusterEtaPhiSpread[k],
         'cscRechitClusterPhi':cscRechitClusterPhi[k],
         'cscRechitClusterEta':np.abs(cscRechitClusterEta[k]),
         'cscRechitClusterZ':np.abs(cscRechitClusterZ[k]),
        }
    else:
        print('BDT NAME NOT FOUND')
    
    if len(cscRechitClusterX[k])>0:
        dataset = pd.DataFrame(bdt_var)
        bdt_score[k] = model.predict_proba(dataset.values)[:, 1]
        print(len(bdt_score[k]) == np.count_nonzero(sel_ev[k]))
        bdt_sel[k] = bdt_score[k] < BDT_CUT 
        if 'data' in k:
            bdt_sel[k] = bdt_score[k] < BDT_CUT 
        else:
            bdt_sel[k] = bdt_score[k] >= BDT_CUT 
        print("effiency",np.count_nonzero(bdt_sel[k])/len(bdt_sel[k]))
    


        dphiMet_cluster[k] = np.abs(T.array('cscRechitClusterMet_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]

    #     jetMet_dPhiMin[k] = T.array('jetMet_dPhiMin')[sel_ev[k]][bdt_sel[k]]
    #     metPhi[k] = T.array('metPhi')[sel_ev[k]]
    #     jetPhi[k] = T.array('jetPhi')[sel_ev[k]]
    #     jetPt[k] = T.array('jetPt')[sel_ev[k]]
    #     jetMet_dPhiMin30[k] = []
    #     for ev in range(len(jetPhi[k])):
    #         dphi_min_temp = 999
    #         for i in range(len(jetPhi[k][ev])):
    #             if jetPt[k][ev][i] < 30: continue
    #             dphi_temp = abs(deltaPhi(metPhi[k][ev],jetPhi[k][ev][i]))
    #             if dphi_min_temp > dphi_temp: dphi_min_temp = dphi_temp
    #         jetMet_dPhiMin30[k].append(dphi_min_temp)
    #     jetMet_dPhiMin30[k] = np.array(jetMet_dPhiMin30[k])
    #     jetMet_dPhiMin30_sr[k] = jetMet_dPhiMin30[k][np.logical_not(bdt_sel[k])]
    #     jetMet_dPhiMin30[k] = jetMet_dPhiMin30[k][bdt_sel[k]]

        weight[k] = T.array('weight')[sel_ev[k]][bdt_sel[k]]
        npv[k] = T.array('npv')[sel_ev[k]][bdt_sel[k]]


        if k[:2] == 'MC':
            pileupWeight[k] = T.array('pileupWeight')[sel_ev[k]][bdt_sel[k]]
        else:
            pileupWeight[k] = np.ones(weight[k].shape, dtype=bool)
        if k[:2] == 'MC':
            weight[k] = weight[k]*lumi[keyToyear(k)]
        cscClusterSize[k] =  T.array('cscRechitClusterSize')[sel_rechitcluster][sel_ev[k]][bdt_sel[k]]
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = cscClusterSize[k][:,0]
    else:
        dphiMet_cluster[k] = np.abs(T.array('cscRechitClusterMet_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()
        weight[k] = T.array('weight')[sel_ev[k]]
        npv[k] = T.array('npv')[sel_ev[k]]
        if k[:2] == 'MC':
            pileupWeight[k] = T.array('pileupWeight')[sel_ev[k]]
        else:
            pileupWeight[k] = np.ones(weight[k].shape, dtype=bool)
        if k[:2] == 'MC':
            weight[k] = weight[k]*lumi[keyToyear(k)]
        cscClusterSize[k] =  T.array('cscRechitClusterSize')[sel_rechitcluster][sel_ev[k]]
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = cscClusterSize[k][:,0]


27570
trigger 310 0.7416220091839738
data 20993309 1258
True
effiency 0.9435612082670907
290
trigger 310 0.9696969696969697
MC_Summer16_ggH_15_100 3069 271
True
effiency 0.9446494464944649
287
trigger 310 0.9528158295281582
MC_Summer16_VBFH_15_100 2628 248
True
effiency 0.9556451612903226
964
trigger 310 0.9801675041876047
MC_Summer16_ggH_15_1000 14925 867
True
effiency 0.9354094579008074
956
trigger 310 0.9455452501405284
MC_Summer16_VBFH_15_1000 14232 763
True
effiency 0.9357798165137615
298
trigger 310 0.9849974374504962
MC_Summer16_ggH_15_10000 21463 280
True
effiency 0.9035714285714286
44
trigger 310 0.9485930735930735
MC_Summer16_VBFH_15_10000 3696 34
True
effiency 0.9411764705882353
44
trigger 310 0.9849843017700323
MC_Summer16_ggH_15_100000 21977 41
True
effiency 0.9024390243902439
11
trigger 310 0.9458425609019528
MC_Summer16_VBFH_15_100000 4967 9
True
effiency 0.7777777777777778
174
trigger 310 0.9507785032646912
MC_Fall17_ggH_15_100 1991 160
True
effiency 0.9625
0
trigger 31

# life time reweight

In [112]:
def weight_calc(llp_ct, new_ctau, old_ctau):
    for i, ct in enumerate(old_ctau):
        if i == 0: source = np.exp(-1.0*llp_ct/ct)/ct**2
        else: source = source + np.exp(-1.0*llp_ct/ct)/ct**2
    weight = len(old_ctau)*1.0/new_ctau**2 * np.exp(-1.0*llp_ct/new_ctau)/source
    return weight

In [31]:
#in cm
# new_ctau = ['5','10','30','40', '50','100','200','300','500','1000', '5000', '10000', '20000','50000', '100000', '200000'. '300000', '500000', '1000000'] #mm
# ctaus = ['100']

# y = 'Summer16'
# m = 15
# one weight, for each year, mass and ctau

# for ctau in ctaus:
#     for m in mass:
#         for y in year:
#             for ct0 in OLD_CTAU:
#                 k = 'MC_'+y+'_ggH_'+str(m)+'_'+str(ct0)
#                 T = tree[k]
#                 new_ctau = float(ctau)
#                 gLLP_ctau[ctau] = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
#                 weight_ctau[ctau] = np.exp(gLLP_ctau[ctau]*(1.0/ct0 - 1.0/new_ctau))*(ct0*1.0/new_ctau)**2
# #         print(ctau, ct0, weight_ctau)




# Signal Contamination

In [41]:
 #in cm
ctaus = ['5','10','30','40', '50','100','200','300','500','1000', '5000', '10000', '100000']
ctaus = ['100']
k = 'mc_signal'
T = tree_bkg['mc']
br = 0.01 # use 1%
DPHI_CUT = 0.6
N_RECHIT_CUTS = np.arange(90,220, 10)

for N_RECHIT_CUT in N_RECHIT_CUTS:
    a_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] >= N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] >= DPHI_CUT))
    b_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] < N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] >= DPHI_CUT))
    c_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] < N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] < DPHI_CUT))
    d_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] >= N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] < DPHI_CUT))
    for ctau in ctaus:
        new_ctau = float(ctau)
        w = br * pileupWeight['mc_vr'] * weight['mc_vr'] * weight_ctau[ctau]
        a = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']>=N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']>=DPHI_CUT)])
        b = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']<N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']>=DPHI_CUT)])
        c = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']<N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']<DPHI_CUT)])
        d = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']>=N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']<DPHI_CUT)])

#         print(N_RECHIT_CUT, a/a_data, a, a_data)
#         print(a,b,c,d)
        w = br * pileupWeight['mc_sr'] * weight['mc_sr'] 
        a = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_sr']>=N_RECHIT_CUT, jetMet_dPhiMin30['mc_sr']>=DPHI_CUT)])
    print(a)
        
        
#         print(N_RECHIT_CUT, a/a_data,b/b_data,c/c_data,d/d_data)

ValueError: operands could not be broadcast together with shapes (14,) (135,) 

# check bdt eff in low N_rechits bins

In [270]:
# k = 'data_intime'
# print("n_rechit, VR, SR, unc")
# for n_rechits in [60,70,80,90]:
#     for dphi in [ 0.4]:
# #         print("dphi, n_rechits", dphi, n_rechits)
#         inverted = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<n_rechits, jetMet_dPhiMin30[k]<dphi))
#         inverted_unc = inverted**0.5/(1-bdtBkgEff)*bdtBkgEff
#         inverted = inverted/(1-bdtBkgEff)*bdtBkgEff
#         print(n_rechits, np.count_nonzero(np.logical_and(nRechits_SR[k]<n_rechits, jetMet_dPhiMin30_sr[k]<dphi)), round(inverted,2), round(inverted_unc,2))

# make datacard

In [284]:
np.arange(120,220, 10)

array([120, 130, 140, 150, 160, 170, 180, 190, 200, 210])

In [116]:
import os

N_RECHIT_CUTS = np.arange(60,170, 10)
# N_RECHIT_CUTS = np.arange(160,170, 10)


# ctaus = ['10','30','40', '50','100','200','300','500','1000', '2000', '3000','5000', '10000', '20000', '30000', '50000','100000']
# ctaus = ['1000']
ctaus = ['5','10','30','40', '50','100','200','300','500','1000', '5000', '10000', '20000','50000', '100000', '200000', '300000', '500000', '1000000'] #mm



# N_RECHIT_CUT = 120
UNBLIND = False
methodA = 1
jetmet = 0
VBF = False
if jetmet: 
    var = jetMet_dPhiMin30
    DPHI_CUT = 0.6
else: 
    var = dphiMet_cluster
    DPHI_CUT = 0.75
eff_path = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/data/bdt_eff/'
eff_file = eff_path + bdt_name +'.txt'
bdtBkgEff_nrechit = np.genfromtxt(eff_file, delimiter=',')
# print(bdtBkgEff_nrechit.shape)
bkg = []
for N_RECHIT_CUT in N_RECHIT_CUTS:
    if methodA:
        ##### method A #####
        row, _ = np.argwhere(bdtBkgEff_nrechit == N_RECHIT_CUT)[0]
        lowN_eff = bdtBkgEff_nrechit[row,1]
        highN_eff = bdtBkgEff_nrechit[row,2]
        k = 'data'
        a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
        b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT))
        c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT))
        d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT))
        if jetmet:
            pred = b*d/c
            stat_unc = (1./b + 1./c + 1./d)**0.5*(pred)
            sys_unc = abs(a-pred)
            unc = [(stat_unc**2 + sys_unc**2)**0.5/pred, b**0.5/b, c**0.5/c, d**0.5/d]
            
        else:
            pred = a*c/b
            stat_unc = (1./a + 1./c + 1./b)**0.5*(pred)
            sys_unc = abs(d-pred)
            unc = [a**0.5/a, b**0.5/b, c**0.5/c, (stat_unc**2 + sys_unc**2)**0.5/pred]
#         print(N_RECHIT_CUT, stat_unc, sys_unc, unc)
        a = a/(1-highN_eff)*highN_eff
        b = b/(1-lowN_eff)*lowN_eff
        c = c/(1-lowN_eff)*lowN_eff
        d = d/(1-highN_eff)*highN_eff    
        pred = pred/(1-highN_eff)*highN_eff
#         print('eff',highN_eff, lowN_eff)
    else:
        ##### method B #####
#         factor = (len(var['data_intime_vr'])+len(var['data_intime_sr']))/(len(var['data_oot_vr'])+len(var['data_oot_sr']))
        factor = (len(var['data_intime_vr']))/(len(var['data_oot_vr']))
#         print('factor', factor, factor2)
        k = 'data_oot_sr'
        a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
        b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT)) 
        c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT)) 
#         print(N_RECHIT_CUT, b_data*d_data/c_data, a_data,  b_data*d_data/c_data*factor)
        if jetmet:
            pred = b*d/c
            stat_unc = (1./b + 1./c + 1./d)**0.5*(pred)
            sys_unc = abs(a-pred)
            unc = [(stat_unc**2 + sys_unc**2)**0.5/pred, b**0.5/b, c**0.5/c, d**0.5/d]
        else:
            pred = a*c/b
            stat_unc = (1./a + 1./c + 1./b)**0.5*(pred)
            sys_unc = abs(d-pred)
            unc = [a**0.5/a, b**0.5/b, c**0.5/c, (stat_unc**2 + sys_unc**2)**0.5/pred]
#         print(N_RECHIT_CUT, stat_unc, sys_unc, unc)
#         print(factor)
        a = a * factor
        b = b * factor
        c = c * factor
        d = d * factor
        pred = pred * factor
    bkg.append(pred)
    #####
    # a = c*c1*c2
    # b = c1* c
    # c = c
    # d = c2*c
    #####
    outDataCardsDir = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/'+\
    ntupler_version+analyzer_version+'/'+bdt_name
    if methodA: outDataCardsDir = outDataCardsDir + '/methodA/'
    else: outDataCardsDir = outDataCardsDir + '/methodB/'
    if not os.path.isdir(outDataCardsDir):os.mkdir(outDataCardsDir)
    sig_norm = []
    for m in mass:
        for ct in ctaus:
            dphi = str(DPHI_CUT).replace(".", "p")
            if VBF: modelName = 'VBFH'
            else: modelName = 'ggh'
            if jetmet:
                modelName = modelName + '_mh125_mx'+str(m)+'_ctau'+str(ct)+'mm_nRechit'+str(N_RECHIT_CUT)+'dPhiJet'+dphi
            else:
                modelName = modelName + '_mh125_mx'+str(m)+'_ctau'+str(ct)+'mm_nRechit'+str(N_RECHIT_CUT)+'dPhiCluster'+dphi
            signal_rate = {}
            signal_rate['ggH'] = np.zeros((4,))
            signal_rate['VBFH'] = np.zeros((4,))
            ctf = int(ct)
            for i, y in enumerate(years):
                if ctf < OLD_CTAU[0]:
                    old_ctau_temp = np.array([OLD_CTAU[0]])
                else:
                    for j, ct0 in enumerate(OLD_CTAU):
                        if ct0 == ctf: 
                            old_ctau_temp = np.array([ctf])
                            break
                        elif ct0 > ctf:
                            old_ctau_temp = np.array([OLD_CTAU[j-1], OLD_CTAU[j]])
                            old_ctau_temp = np.array([OLD_CTAU[j]])
                            break
                        if j == len(OLD_CTAU)-1: 
                            old_ctau_temp = np.array([OLD_CTAU[j]])
                weight_sum = 0
                weight_len = 0
                for j,ct0 in enumerate(old_ctau_temp):
                    if VBFH: prods = ['ggH', 'VBFH']
                    else: prods = ['ggH']
                    for prod in prods:
                        k = 'MC_'+y+'_'+prod+'_'+str(m)+'_'+str(ct0)
                        T = tree[k]
                        if np.count_nonzero(sel_ev[k]) == 0: continue
                        gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                        if len(gLLP_ctau) == 0: continue
                        weight_ctau = weight_calc(gLLP_ctau, float(ct)/10, old_ctau_temp/10) # convert everything to cm
                        w = pileupWeight[k] * weight[k] * weight_ctau
                        weight_sum +=np.sum(weight_ctau)
                        weight_len += len(weight_ctau)                    
                        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]>=DPHI_CUT)
                        signal_rate[prod][0]+=np.sum(w[cond])
                        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]>=DPHI_CUT)
                        signal_rate[prod][1]+=np.sum(w[cond])
                        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]<DPHI_CUT)
                        signal_rate[prod][2]+=np.sum(w[cond])
                        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT)
                        signal_rate[prod][3]+=np.sum(w[cond])
                    
            norm = np.sum(signal_rate['VBFH']+signal_rate['ggH'])/2
            if norm == 0: continue
            sig_norm.append(norm)
#             print(m, ct, norm, weight_sum/weight_len)
            signal_rate = signal_rate/norm
            if VBF:
                make_datacard_2sig(outDataCardsDir,modelName, jetmet, signal_rate['ggH'], signal_rate['VBFH'], norm, a,b,c,d, unc)
            else:
                make_datacard(outDataCardsDir, modelName, jetmet, signal_rate['ggH'], norm, pred,b,c,d, unc)
#     print(sig_norm)
print(bkg)

[13.679829084176044, 10.663286384976526, 7.478655909767369, 5.609539746848185, 3.5192494338401814, 1.7881745120551087, 1.824635265912347, 1.5550510783200906, 0.929054054054054, 0.8716936439005131, 1.0069930069930069]


In [ ]:
15 1000 1938.8570868046945
40 1000 90.73068838176704
55 1000 3.2566280769474663
[1.0069930069930069]

In [115]:
def make_datacard(outDataCardsDir,modelName, jetmet, signal_rate, norm, a,b,c,d, unc):
    if jetmet:
        c1 = b/c
        c2 = d/c
    else:
        c1 = a/b
        c2 = c/b
    text_file = open(outDataCardsDir+modelName+".txt", "w")
    text_file.write('# signal norm {0} \n'.format(norm))

    text_file.write('imax {0} \n'.format(4))
    text_file.write('jmax {0} \n'.format(1))
    text_file.write('kmax * \n')
    text_file.write('--------------- \n')
    text_file.write('--------------- \n')
    text_file.write('bin \t chA \t chB \t chC \t chD \n')
    text_file.write('observation \t {0:6.2f} \t {1:6.2f} \t {2:6.2f} \t {3:6.2f} \n'.format(a, b, c, d))
    text_file.write('------------------------------ \n')  
    text_file.write('bin \t chA \t chA \t chB \t chB \t chC \t chC \t chD \t chD \n')
    text_file.write('process sig \t bkg \t sig \t bkg \t sig \t bkg \t sig \t bkg \n')
    text_file.write('process 0 \t 1 \t 0 \t 1 \t 0 \t 1 \t 0 \t 1 \n')
    rate_string = 'rate'
    for i, rate in enumerate(signal_rate):

        rate_string =  rate_string+'\t {0:e} \t 1'.format(rate)
    text_file.write(rate_string+'\n')
    text_file.write('------------------------------ \n')  
    for ch in ['A','B','C','D']:
        text_file.write('NA\t rateParam \t ch{0} \t bkg \t {1:6.2f} \n'.format(ch, c))
    if jetmet:
        text_file.write('c1\t rateParam \t chA \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chA \t bkg \t {0:e} \n'.format(c2))
        text_file.write('c1\t rateParam \t chB \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chD \t bkg \t {0:e} \n'.format(c2))
    else:
        text_file.write('c1\t rateParam \t chD \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chD \t bkg \t {0:e} \n'.format(c2))
        text_file.write('c1\t rateParam \t chA \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chC \t bkg \t {0:e} \n'.format(c2))
#     #### uncertainties ####
    text_file.write('lumi\t lnN \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \n')
    text_file.write('signal_yield \t lnN \t 1.2 \t - \t 1.2 \t - \t 1.2 \t - \t 1.2 \t - \n')
    text_file.write('data_pred \t lnN \t - \t {0} \t - \t {1} \t - \t {2} \t - \t {3} \n'.format(1+unc[0], 1+unc[1], 1+unc[2], 1+unc[3]))



    text_file.close()


In [90]:
def make_datacard_2sig(outDataCardsDir,modelName, jetmet, signal_rate1, signal_rate2, norm, a,b,c,d, unc):
    if jetmet:
        c1 = b/c
        c2 = d/c
    else:
        c1 = a/b
        c2 = c/b
    
    text_file = open(outDataCardsDir+modelName+".txt", "w")
    text_file.write('# signal norm {0} \n'.format(norm))

    text_file.write('imax {0} \n'.format(4))
    text_file.write('jmax {0} \n'.format(1))
    text_file.write('kmax * \n')
    text_file.write('--------------- \n')
    text_file.write('--------------- \n')
    text_file.write('bin \t chA \t chB \t chC \t chD \n')
    text_file.write('observation \t {0:6.2f} \t {1:6.2f} \t {2:6.2f} \t {3:6.2f} \n'.format(a, b, c, d))
    text_file.write('------------------------------ \n')  
    text_file.write('bin \t chA \t chA \t chA \t chB \t chB \t chB \t chC \t chC \t chC \t chD \t chD \t chD \n')
    text_file.write('process ggH \t VBFH \t bkg \t ggH \t VBFH \t bkg \t ggH \t VBFH \t bkg \t ggH \t VBFH \t bkg \n')
    text_file.write('process 0 \t 0 \t 1 \t 0 \t 0 \t 1 \t 0 \t 0 \t 1 \t 0 \t 0 \t 1 \n')
    rate_string = 'rate'
    for i, rate in enumerate(signal_rate1):
        rate_string =  rate_string+'\t {0:e} \t {1:e} \t1'.format(rate, signal_rate2[i])
    text_file.write(rate_string+'\n')
    text_file.write('------------------------------ \n')  
    for ch in ['A','B','C','D']:
        text_file.write('NA\t rateParam \t ch{0} \t bkg \t {1:6.2f} \n'.format(ch, c))

    if jetmet:
        text_file.write('c1\t rateParam \t chA \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chA \t bkg \t {0:e} \n'.format(c2))
        text_file.write('c1\t rateParam \t chB \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chD \t bkg \t {0:e} \n'.format(c2))
    else:
        text_file.write('c1\t rateParam \t chD \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chD \t bkg \t {0:e} \n'.format(c2))
        text_file.write('c1\t rateParam \t chA \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chC \t bkg \t {0:e} \n'.format(c2))

#     #### uncertainties ####
    text_file.write('lumi\t lnN \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \n')
    text_file.write('signal_yield \t lnN \t 1.2 \t - \t 1.2 \t - \t 1.2 \t - \t 1.2 \t - \n')
    text_file.write('data_pred \t lnN \t - \t {0} \t - \t {1} \t - \t {2} \t - \t {3} \n'.format(1+unc[0], 1+unc[1], 1+unc[2], 1+unc[3]))



    text_file.close()


# significance optimization

# background prediction

# create datacard

# check correlation

## Distribution for event level variables in different binnings

# Roc curve for event level variables

# Data plot

In [ ]:
import importlib
importlib.reload(sys.modules['histo_utilities'])
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
leg = rt.TLegend(0.5,0.80,0.90,0.90)

leg.SetTextSize(0.03)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
c = rt.TCanvas('c','c', 800, 800)
# Plotting ncsc
start_t = time.time()

legend = {}
legend['data_oot'] = 'Data OOT region'
legend['data_intime'] = 'Data in-time region'
legend['mc_signal'] = 'signal MC in time'
legend['mc_intime'] = 'MC in-time background'
legend['mc_oot'] = 'MC OOT background'
legend['mc_bkg'] = 'QCD 50toInf background'




keys = ['data_intime','data_oot','mc_signal',]
keys = ['data_intime','mc_signal',]


h = {}
rt.gStyle.SetOptFit(1011)
# for i,k in enumerate(tree_bkg.keys()):
br = 1
for i,k in enumerate(keys):

#     if k == 'data_intime':
#         cond = nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < 200
        
#     else:
#         cond = np.ones(weight[k].shape, dtype=bool)
#     cond = np.logical_and(cond, jetMet_dPhiMin4[k]>0.25)
    if k == 'mc_signal':
        br = 1
    else: br = 1
#     if k == 'mc_background':br = 1
    bins = [50,0,1000]
    metcut = 200
    h[k] = create_TH1D( nCsc_JetMuonVetoCluster0p4_Me1112Veto[k][cond], axis_title=['N_{rechits}', 'Events'], name=k, binning=bins,weights = br*weight[k][cond])
    h[k].SetLineColor(std_color_list[i])
    print("weight",weight[k][0])
    leg.AddEntry(h[k], legend[k]+" (scaled)" if k == 'data_oot' else legend[k])
#     if k == 'mc_signal':
#         for i in range(h[k].GetNbinsX()):
#             s = 0.015*h[k].GetBinContent(i+1)
#             b = h['data_oot'].GetBinContent(i+1)
#             if s+b == 0:continue
#             sig = s/(s+b)**0.5
#             print(i,h[k].GetBinCenter(i+1),sig,s,b)
#             if sig < 1./100:
#                 print(i,sig)
#                 continue
#     h[k].SetMinimum(1)

#     r = h[k].Fit('expo', 'LRSQ+', '', 50,400)
#     print(r.Get())
#     print(r.Prob())
#     h[k].GetFunction("expo").SetLineColor(std_color_list[2])
#     h[k].GetFunction("expo").SetLineWidth(2)
    h[k].GetXaxis().SetLabelSize(0.04)
    h[k].SetMinimum(0.1)
    h[k].SetMaximum(10E6/2)
#     h[k+'met'].GetXaxis().SetLabelSize(0.02)
    if k == 'data_oot':
        scale = 1.0*h['data_intime'].GetBinContent(4)/h['data_oot'].GetBinContent(4)
#         for i in range(10):
#             print(1.0*h['data_intime'].GetBinContent(i+1)/h['data_oot'].GetBinContent(i+1))
        h[k].Scale(scale)
    if k[:4] == 'data':
        h[k].Draw('same E1')
    else:
        h[k].Draw("same hist")
#     h[k+'met'].DrawNormalized('same')
    

# c = make_ratio_plot([h['mc'],h['data']], fit = False, logy=True, in_tags = ["MC","Data"], ratio_bounds = [0,3])
c.SetRightMargin(0)
c.SetLogy()
leg.Draw()

c.Draw()
# print(time.time()-start_t)

In [ ]:
import importlib
importlib.reload(sys.modules['histo_utilities'])
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
leg = rt.TLegend(0.5,0.80,0.90,0.90)

leg.SetTextSize(0.03)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
c = rt.TCanvas('c','c', 800, 800)
# Plotting ncsc
start_t = time.time()


k = 'data_intime'

h = {}
rt.gStyle.SetOptFit(1011)

bins = [40,0,1000]
metcut = 200

# cscRechitClusterMaxChamber = {}
# cscRechitClusterNChamber = {}
# cscRechitClusterNStation = {}
# cscRechitClusterMaxStationRatio = {}
# cscRechitClusterMaxChamberRatio = {}

# h[k] = create_TH1D( cscRechitClusterMaxChamber[k], axis_title=['max chamber', 'Events'], name=k, binning=[200,-50,50])
h[k] = create_TH1D( cscRechitClusterMaxChamberRatio[k], axis_title=['max chamber ratio', 'Events'], name=k, binning=[20,0,1])
# h[k] = create_TH1D( cscClusterMet_dPhi[k], axis_title=['#Delta#phi(MET, cluster)', 'Events'], name=k, binning=[20,0,3.14])
# h[k] = create_TH1D( n_chamber[k], axis_title=['Number of Chambers', 'Events'], name=k, binning=[10,0,10])
# h[k] = create_TH1D( cscRechitClusterNStation[k], axis_title=['max station', 'Events'], name=k, binning=[5,0,5])
# print(cscRechitClusterMaxStationRatio[k])
# h[k] = create_TH1D( cscRechitClusterNStation[k], axis_title=['Number of station', 'Events'], name=k, binning=[20,0,10])


# h['1jet'].GetXaxis().SetLabelSize(0.04)
# h['1jet'].SetMinimum(0.1)
# h['1jet'].SetMaximum(10E6/2)

h[k].Draw('hist')


c.SetRightMargin(0)
c.SetLogy()

c.Draw()
# print(time.time()-start_t)

## significance

In [ ]:
start_t = time.time()
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.35,0.80,0.85,0.92)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)

var = [nCsc_JetMuonVetoCluster0p4_Me1112Veto,]
name = ['nCsc_JetMuonVetoCluster0p4_Me1112Veto',]
legend = [ 'jet & muon veto + ME11/12 veto',]

bkg_k = 'data_oot'
sig_k = 'mc_signal'
br = 0.01

threshold = list(range(300))
threshold = np.array(list(range(2000)))
# print(threshold)
for i in range(len(var)):
    event_count = {}
    
    for k in ['data_oot','mc_signal']:
        event_count[k] = []
        for th in threshold:
            event_count[k].append(np.sum(weight[k][var[i][k]>th]))
        event_count[k] = np.array(event_count[k])
        if k == sig_k:
            event_count[k] = event_count[k]*br
    sig = event_count[sig_k]/np.sqrt(event_count[sig_k]+event_count[bkg_k])
    cond = event_count[sig_k]>0
    sig = sig[cond]
    ncsc = threshold[cond]
#     gr['bbbb'] = create_TGraph(eff_bkg,eff_sig,axis_title = ['#epsilon_{bkg}','#epsilon_{signal}'])
    h[sig_k+str(i)] = create_TGraph(ncsc, sig,  axis_title=['N_{rechits}', 'Significnace'])
#     h[sig_k+str(i)] = create_TGraph(ncsc, event_count[sig_k],  axis_title=['N_{rechits}', 'Event Yield'])

    h[sig_k+str(i)].SetLineWidth(2)
    h[sig_k+str(i)].SetMarkerColor(std_color_list[i])
    h[sig_k+str(i)].SetLineColor(std_color_list[i])
    h[sig_k+str(i)].GetXaxis().SetRangeUser(0, 3000)
    h[sig_k+str(i)].GetXaxis().SetLabelSize(0.04)

#     h[sig_k+str(i)].GetYaxis().SetRangeUser(0, 3)


    leg.AddEntry(h[sig_k+str(i)],sig_k+" "+legend[i])
    h[sig_k+str(i)].Draw('ac' if i == 0 else 'c')

#     h[sig_k+str(i)].Draw('ac' if i==0 else 'c')
#     gr['bbbb'].Draw('')




# c.SetLogy()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')

print(time.time()-start_t)


# 2d plot

In [ ]:
# Plotting ncsc
start_t = time.time()
c = rt.TCanvas('c','c', 1000, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.94,0.85)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
k = 'data_intime'

# h[k] = create_TH2D(np.column_stack((angle[k][cond],jetPt[k][cond][:,0])), axis_title = ['\Delta\phi (jet,met)','jet p_{T}[GeV]','Events'], binning=[80,-3.14,3.14,100,0,2000])
h[k] = create_TH2D(np.column_stack(( nCsc_JetMuonVetoCluster0p4_Me1112Veto[k], dphiMet_cluster[k])),
                   axis_title = ['N_{rechits}','\Delta\phi (cluster,met)','Events'], binning=[50,0,600,50,0,3.14])
h[k] = create_TH2D(np.column_stack(( nCsc_JetMuonVetoCluster0p4_Me1112Veto[k], jetMet_dPhiMin4[k])),
                   axis_title = ['N_{rechits}','min_{4jet}\Delta\phi (jet,met)','Events'], binning=[50,0,600,50,0,3.14])


h[k].GetXaxis().SetLabelSize(0.04)




h[k].SetLineColor(std_color_list[i])
# leg.AddEntry(h[k], k)
#     h[k].SetMaximum(10E5)
#     h[k].SetMinimum(1)
h[k].Draw('colz')
c.SetRightMargin(0.2)



# if setlog: 
c.SetLogz()
c.Draw()
print(time.time()-start_t)

# third

# ROC Curve

# 4th

# nCsc histogram

# event yield vs. ncsc

# significance